In [1]:
# Model training of text classiffiers using logreg, svm, mlp 
# Violeta Berdejo-Espinola, Akos Hajas, Nan Ye
# November 2024

In [ ]:
%pip install mpu scikit-learn imblearn embetter sentence_transformers matplotlib ipywidgets

# read data

In [3]:
import mpu

# raw corpus

corpus_raw = mpu.io.read('../data/corpus_raw.pickle')
corpus_raw_long = mpu.io.read('../data/corpus_raw_long.pickle')

x_raw = corpus_raw
x_raw_long = corpus_raw_long

# clean corpus

corpus = mpu.io.read('../data/corpus_clean.pickle')
corpus_long = mpu.io.read('../data/corpus_clean_long.pickle')

x = corpus
x_long = corpus_long

# pos, negs

pos = mpu.io.read('../data/pos.pickle')
neg = mpu.io.read('../data/neg.pickle')
y = [1] * len(pos) + [0] * len(neg)

In [4]:
weight_for_class_0 = len(x) / (len(neg) * 2) 
weight_for_class_1 = len(x) / (len(pos) * 2) 

# split data

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
x_train_long, x_test_long, y_train_long, y_test_long = train_test_split(x_long, y, test_size=0.20, random_state=42)
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(x_raw, y, test_size=0.20, random_state=42)
x_train_r_long, x_test_r_long, y_train_r_long, y_test_r_long = train_test_split(x_raw_long, y, test_size=0.20, random_state=42)

In [ ]:
from collections import Counter

counter = Counter()

for i in y_test:
    counter[i] +=1
    
print(counter)

# instantiate feature extractors, embedding models, resamplers, models 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer # uses one-dim array of strings ~ shape (n,)
from sklearn.feature_extraction.text import CountVectorizer # returns arrays

vect_cv = CountVectorizer()
vect_tfidf = TfidfVectorizer()

model_mpnet = 'paraphrase-multilingual-mpnet-base-v2'
model_distill = 'distiluse-base-multilingual-cased-v1'

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN 

rus = RandomUnderSampler(random_state=42, sampling_strategy=1)
ros = RandomOverSampler(random_state=42, sampling_strategy='not majority')
ada = ADASYN(random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

logreg = LogisticRegression(solver='liblinear', random_state=42)
logreg_weight = LogisticRegression(solver='liblinear', class_weight={0: weight_for_class_0, 1: weight_for_class_1}, random_state=42)
svm = SVC(kernel='linear')
svm_weight = SVC(kernel='linear', class_weight={0: weight_for_class_0, 1: weight_for_class_1}, probability=True)
mlp = MLPClassifier(activation='logistic', batch_size=16, hidden_layer_sizes=(), learning_rate='constant',learning_rate_init=0.001, solver='adam', random_state=42)
mlp_t = MLPClassifier(activation='logistic', batch_size=16, hidden_layer_sizes=(5,), learning_rate='invscaling',learning_rate_init=1, solver='sgd', random_state=42, max_iter=400)

# function to train eval models

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline 

from embetter.text import SentenceEncoder

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score, precision_score, recall_score

import pandas as pd

In [9]:
# TF TFIDF baseline and weighted

def train_eval_tf_tfidf(x_train, y_train, x_test, y_test, text_length, kfold):
    
    def run_estimator(estimator, feature_extractor, balanced):
        
        pipeline = Pipeline([("vectorizer", feature_extractor),
                             ("estimator", estimator)
                            ])
    
        y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict') # cross val splits the data and then applies the pipeline steps

        pipeline.fit(x_train, y_train)
        
        y_train_pred = pipeline.predict(x_train)

        y_test_pred = pipeline.predict(x_test)

        return {
            'Classifier': pipeline['estimator'],
            'Feature_extraction': pipeline['vectorizer'],
            'Weighting': 'Weighted' if balanced else 'None',
            'CV': kfold,
            'Text_length': text_length,
            'F1_tr_cv': round(f1_score(y_train, y_train_pred_cv), 3),
            'F1_tr': round(f1_score(y_train, y_train_pred), 3),
            'F1_ts': round(f1_score(y_test, y_test_pred), 3),
            'Precision_tr_cv': round(precision_score(y_train, y_train_pred_cv), 3),
            'Precision_tr': round(precision_score(y_train, y_train_pred), 3),
            'Precision_ts': round(precision_score(y_test, y_test_pred), 3),
            'Recall_tr_cv': round(recall_score(y_train, y_train_pred_cv), 3),
            'Recall_tr': round(recall_score(y_train, y_train_pred), 3),
            'Recall_ts': round(recall_score(y_test, y_test_pred), 3)
        }


    all_scores = []
    
    for feature_extractor in [vect_cv, vect_tfidf]:
        
        for estimator in [logreg, svm]:
            all_scores.append(run_estimator(estimator, feature_extractor, False))
            
        for estimator in [logreg_weight, svm_weight]: 
            all_scores.append(run_estimator(estimator, feature_extractor, True))

    return all_scores

In [10]:
# embeddings baseline and weighted

def train_eval_embeddings(x_train, y_train, x_test, y_test, text_length, kfold):
    
    def run_estimator(estimator, balanced):
        
        pipeline = make_pipeline(
            SentenceEncoder(embedding_model),
            estimator
        )
    
        y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict')

        pipeline.fit(x_train, y_train)
        
        y_train_pred = pipeline.predict(x_train)

        y_test_pred = pipeline.predict(x_test)

        return {
            'Classifier': estimator,
            'Feature_extraction': embedding_model,
            'Weighting': 'Weighted' if balanced else None,
            'CV': kfold,
            'Text_length': text_length,
            'F1_tr_cv': round(f1_score(y_train, y_train_pred_cv), 3),
            'F1_tr': round(f1_score(y_train, y_train_pred), 3),
            'F1_ts': round(f1_score(y_test, y_test_pred), 3),
            'Precision_tr_cv': round(precision_score(y_train, y_train_pred_cv), 3),
            'Precision_tr': round(precision_score(y_train, y_train_pred), 3),
            'Precision_ts': round(precision_score(y_test, y_test_pred), 3),
            'Recall_tr_cv': round(recall_score(y_train, y_train_pred_cv), 3),
            'Recall_tr': round(recall_score(y_train, y_train_pred), 3),
            'Recall_ts': round(recall_score(y_test, y_test_pred), 3)
        }


    all_scores = []
    
    for embedding_model in [model_mpnet, model_distill]:
        
            for estimator in [logreg, svm, mlp, mlp_t]:
                all_scores.append(run_estimator(estimator, False))
                
            for estimator in [logreg_weight, svm_weight]: 
                all_scores.append(run_estimator(estimator, True))

    return all_scores

In [11]:
# TF TFIDF resampled

def train_eval_tf_tfidf_resampled(x_train, y_train, x_test, y_test, text_length, kfold):
    
    all_scores = []
    
    for feature_extractor in [vect_cv, vect_tfidf]:
        
        for resampler in [rus, ros, ada]:
        
            for estimator in [logreg, svm, mlp]:
        
                pipeline = Pipeline([("vectorizer", feature_extractor),
                                    ("resampler", resampler),
                                    ("estimator", estimator)
                                    ])
        
                y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict')

                pipeline.fit(x_train, y_train)
                
                y_train_pred = pipeline.predict(x_train)

                y_test_pred = pipeline.predict(x_test)

                scores = {
                    'Classifier': pipeline['estimator'],
                    'Feature_extraction': pipeline['vectorizer'],
                    'Weighting': pipeline['resampler'],
                    'CV': kfold,
                    'Text_length': text_length,
                    'F1_tr_cv': round(f1_score(y_train, y_train_pred_cv), 3),
                    'F1_tr': round(f1_score(y_train, y_train_pred), 3),
                    'F1_ts': round(f1_score(y_test, y_test_pred), 3),
                    'Precision_tr_cv': round(precision_score(y_train, y_train_pred_cv), 3),
                    'Precision_tr': round(precision_score(y_train, y_train_pred), 3),
                    'Precision_ts': round(precision_score(y_test, y_test_pred), 3),
                    'Recall_tr_cv': round(recall_score(y_train, y_train_pred_cv), 3),
                    'Recall_tr': round(recall_score(y_train, y_train_pred), 3),
                    'Recall_ts': round(recall_score(y_test, y_test_pred), 3)    
                }
    
                all_scores.append(scores)

    return all_scores

In [12]:
# embeddings resampled

def train_eval_embedding_resampled(x_train, y_train, x_test, y_test, text_length, kfold, embed_model):
    
    all_scores = []
    
    pipeline = make_pipeline(
        SentenceEncoder(embed_model),
        RandomOverSampler(random_state=42, sampling_strategy='not majority'),
        MLPClassifier(activation='logistic', batch_size=16, hidden_layer_sizes=(), learning_rate='constant',learning_rate_init=0.001, solver='adam', random_state=42)
    )
                                
    y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict')

    pipeline.fit(x_train, y_train)
    
    y_train_pred = pipeline.predict(x_train)

    y_test_pred = pipeline.predict(x_test)

    scores = {
        'Classifier': mlp,
        'Feature_extraction': embed_model,
        'Weighting': ros,
        'CV': kfold,
        'Text_length': text_length,
        'F1_tr_cv': round(f1_score(y_train, y_train_pred_cv), 3),
        'F1_tr': round(f1_score(y_train, y_train_pred), 3),
        'F1_ts': round(f1_score(y_test, y_test_pred), 3),
        'Precision_tr_cv': round(precision_score(y_train, y_train_pred_cv), 3),
        'Precision_tr': round(precision_score(y_train, y_train_pred), 3),
        'Precision_ts': round(precision_score(y_test, y_test_pred), 3),
        'Recall_tr_cv': round(recall_score(y_train, y_train_pred_cv), 3),
        'Recall_tr': round(recall_score(y_train, y_train_pred), 3),
        'Recall_ts': round(recall_score(y_test, y_test_pred), 3)    
    }

    all_scores.append(scores)

    return all_scores

# train eval models

In [13]:
import pandas as pd

In [14]:
# TF TFIDF baseline and weighted

df1 = pd.DataFrame(train_eval_tf_tfidf(x_train, y_train, x_test, y_test,'Title_Abstract',2))
df2 = pd.DataFrame(train_eval_tf_tfidf(x_train_long, y_train_long, x_test_long, y_test_long,'Title_Abstract_Main',2))

In [15]:
# embeddings baseline and weighted

df3 = pd.DataFrame(train_eval_embeddings(x_train_r, y_train_r, x_test_r, y_test_r,'Title_Abstract',2))
df4 = pd.DataFrame(train_eval_embeddings(x_train_r_long, y_train_r_long, x_test_r_long, y_test_r_long,'Title_Abstract_Main',2))

In [16]:
# TF TFIDF resampled

df5 = pd.DataFrame(train_eval_tf_tfidf_resampled(x_train, y_train, x_test, y_test,'Title_Abstract',2))
df6 = pd.DataFrame(train_eval_tf_tfidf_resampled(x_train_long, y_train_long, x_test_long, y_test_long,'Title_Abstract_Main',2))

In [17]:
# embeddings resampled

df7 = pd.DataFrame(train_eval_embedding_resampled(x_train_r, y_train_r, x_test_r, y_test_r,'Title_Abstract',2, model_mpnet))
df8 = pd.DataFrame(train_eval_embedding_resampled(x_train_r_long, y_train_r_long, x_test_r_long, y_test_r_long,'Title_Abstract_Main',2, model_mpnet))

df9 = pd.DataFrame(train_eval_embedding_resampled(x_train_r, y_train_r, x_test_r, y_test_r,'Title_Abstract',2, model_distill))
df10 = pd.DataFrame(train_eval_embedding_resampled(x_train_r_long, y_train_r_long, x_test_r_long, y_test_r_long,'Title_Abstract_Main',2, model_distill))

# cocatenate model results

In [18]:
res = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])
res = res.sort_values(by='Recall_tr_cv', ascending=False).reset_index(drop=True)
res 

In [19]:
# res.to_csv('../results/preliminary/model_results_dec2024_embedd.csv')

# best performing model

In [20]:
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

# predict probabilities and calculate model loss
# log_likelihood = y_test*np.log(y_pred) + (1-y_test)*np.log(1-y_pred)

In [21]:
def train_eval_best_model(x_train, y_train, x_test, y_test, text_length, kfold, embedding_model, random_state):
    
    pipeline = make_pipeline(
        SentenceEncoder(embedding_model),
        LogisticRegression(solver='liblinear', class_weight={0: weight_for_class_0, 1: weight_for_class_1}, random_state=random_state)
    )

    y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict') 

    pipeline.fit(x_train, y_train)
    
    y_train_pred = pipeline.predict(x_train)

    y_test_pred = pipeline.predict(x_test)

    scores = {
        'Classifier': 'Log_reg',
        'Feature_extraction': embedding_model,
        'Weighting': 'Weighted',
        'CV': kfold,
        'Text_length': text_length,
        'Solver': 'Liblinear',
        'F1_tr_cv': round(f1_score(y_train, y_train_pred_cv), 3),
        'F1_tr': round(f1_score(y_train, y_train_pred), 3),
        'F1_ts': round(f1_score(y_test, y_test_pred), 3),
        'Precision_tr_cv': round(precision_score(y_train, y_train_pred_cv), 3),
        'Precision_tr': round(precision_score(y_train, y_train_pred), 3),
        'Precision_ts': round(precision_score(y_test, y_test_pred), 3),
        'Recall_tr_cv': round(recall_score(y_train, y_train_pred_cv), 3),
        'Recall_tr': round(recall_score(y_train, y_train_pred), 3),
        'Recall_ts': round(recall_score(y_test, y_test_pred), 3)
        }
    
    # train set
    y_pred_tr = pipeline.predict_proba(x_train) # predict_proba returns probabilities of a classification label
    logloss_tr = log_loss(y_train, y_pred_tr) 

    # test set
    y_pred_ts = pipeline.predict_proba(x_test)  
    logloss_ts = log_loss(y_test, y_pred_ts)
    
    print(f'loss training set: {logloss_tr}, \nloss test set {logloss_ts}')
    print(f'confusion matrix train set:\n{confusion_matrix(y_train, pipeline.predict(x_train))}')
    print(f'confusion matrix test set:\n{confusion_matrix(y_test, pipeline.predict(x_test))}') 
 
    return scores, y_pred_ts, logloss_tr, logloss_ts

In [ ]:
# best model 

best_model = train_eval_best_model(x_train_r, y_train_r, x_test_r, y_test_r, 'Title_Abstract', 2, model_distill, 42)
best_model

# investigating performance on different train-test partitions

In [23]:
# run on a few different train-test partitions, then report the average with the standard error.
# I should see greater performance with more data, but also lower variance across the different random samples

In [ ]:
m_seed24 = train_eval_best_model(x_train_r, y_train_r, x_test_r, y_test_r, 'Title_Abstract', 2, model_distill, 24)
m_seed36 = train_eval_best_model(x_train_r, y_train_r, x_test_r, y_test_r, 'Title_Abstract', 2, model_distill, 36)
m_seed64 = train_eval_best_model(x_train_r, y_train_r, x_test_r, y_test_r, 'Title_Abstract', 2, model_distill, 64)
m_seed128 = train_eval_best_model(x_train_r, y_train_r, x_test_r, y_test_r, 'Title_Abstract', 2, model_distill, 128)

m_seeds = [m_seed24[2], m_seed36[2], m_seed64[2], m_seed128[2], best_model[2]]

from statistics import stdev

std_dev = stdev(m_seeds)
std_dev # logloss


# plot predicted probabilities


In [ ]:
import matplotlib.pyplot as plt

y_pred_ts_p = best_model[1][:,1] # subset class of interest --> 1d array
y_pred_ts_n = best_model[1][:,0]

plt.figure(figsize=(15,7))
plt.hist(y_pred_ts_n, bins=50, label='Negatives')
plt.hist(y_pred_ts_p, bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Predicted probabilities', fontsize=15)
plt.ylabel('Number of examples', fontsize=15)
plt.legend(fontsize=15)
plt.tick_params(axis='both', labelsize=15, pad=5)
plt.show() 
# plt.savefig('../results/logreg_svm_mlp/predicted_proba_logreg_ts.png', dpi=800)

# roc


In [26]:
# function to predict probabilities, compute auc score and roc curve 

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

def get_auc_roc(estimator, embedding_model, x_train, y_train, x_test, y_test, random_state):
    
    pipeline = make_pipeline(
        SentenceEncoder(embedding_model),
        LogisticRegression(solver='liblinear', class_weight={0: weight_for_class_0, 1: weight_for_class_1}, random_state=random_state)
    )
    
    # y_train_pred_cv = cross_val_predict(pipeline, x_train, y_train, cv=StratifiedKFold(kfold), method='predict') 

    pipeline.fit(x_train, y_train)
    
    pred_prob = pipeline.predict_proba(x_test)
    y_pred = pipeline.predict(x_test)

    auc_score = roc_auc_score(y_test, pred_prob[:,1])
    fpr, tpr, thresh = roc_curve(y_test, pred_prob[:,1], pos_label=1)
    
    return auc_score, fpr, tpr, pred_prob, y_pred

# roc curve for tpr = fpr 

random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

auc_lr_weight_distill, fpr_lr_weight_distill, tpr_lr_weight_distill, y_pred_proba_lr_weight_distill, y_pred_lr_weight__distill= get_auc_roc(logreg_weight, model_distill, x_train_r, y_train_r, x_test_r, y_test_r, 42)
auc_lr_weight_mpnet, fpr_lr_weight_mpnet, tpr_lr_weight_mpnet, y_pred_proba_lr_weight_mpnet, y_pred_lr_weight_mpnet= get_auc_roc(logreg_weight, model_mpnet, x_train_r, y_train_r, x_test_r, y_test_r, 42)
auc_svm_weight_distill, fpr_svm_weight_distill, tpr_svm_weight_distill, y_pred_proba_svm_weight_distill, y_pred_svm_weight_distill= get_auc_roc(svm_weight, model_distill, x_train_r, y_train_r, x_test_r, y_test_r, 42)
auc_svm_weight_mpnet, fpr_svm_weight_mpnet, tpr_svm_weight_mpnet, y_pred_proba_svm_weight_mpnet, y_pred_svm_weight_mpnet= get_auc_roc(svm_weight, model_mpnet, x_train_r, y_train_r, x_test_r, y_test_r, 42)


In [ ]:
import matplotlib.pyplot as plt
# plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr_lr_weight_distill, tpr_lr_weight_distill, linestyle='--',color='#bbca', label='Logistic Regression - weighted - distill embeddings')
plt.plot(fpr_svm_weight_distill, tpr_svm_weight_distill, linestyle='--',color='blue', label='SVM - weighted - distill embeddings')
plt.plot(fpr_lr_weight_mpnet, tpr_lr_weight_mpnet, linestyle='--',color='gray', label='Logistic Regression - weighted - mpnet embeddings')
plt.plot(fpr_svm_weight_mpnet, tpr_svm_weight_mpnet, linestyle='--',color='red', label='SVM - weighted - mpnet embeddings')

plt.plot(p_fpr, p_tpr, linestyle='--', color='gray')

# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('../results/ROC.png',dpi=800)
plt.show();

# error analysis

In [28]:
# combine x df with df containing text to explore 
# missclassification of false positive instances 

In [ ]:
# log reg

x_test_df = pd.DataFrame(x_test)

dat_pred_label = pd.DataFrame(best_model[1], columns=['neg_label', 'pos_label'])
dat_pred_label['y_true'] = y_test
dat_pred_label

# assign predicted labels to examples

def get_prediction_label(row):
    if row["y_true"] == 0:
        return 'fp' if row['neg_label'] < 0.5 else 'tn'
    else:
        return 'tp' if row['pos_label'] > 0.5 else 'fn'

dat_pred_label['prediction_label'] = dat_pred_label.apply(get_prediction_label, axis=1)

dat_pred_label = dat_pred_label.merge(x_test_df, left_index=True, right_index=True)
# dat_pred_label.to_csv('../results/preliminary/error_analysis_predictions_with_tetx.csv')

fp_examples = dat_pred_label[dat_pred_label['prediction_label'] == 'fp']
fp_examples
